# Modelisation supervisee

## A. Pre-requis et Nettoyage

In [41]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import re

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [42]:
df = pd.read_csv("dataset/dataset-clean-v2.csv")

In [43]:
df.head()

,Unnamed: 0,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,preproc_tags,preproc_body,preproc_title
0,0,Is gettimeofday() guaranteed to be of microsec...,"<p>I am porting a game, that was originally wr...",<linux><winapi><visual-c++><unix><timer>,88,107,45385,10,"['linux', 'winapi', 'visual-c++', 'unix', 'tim...","['port', 'game', 'originally', 'write', 'win32...","['gettimeofday', 'guarantee', 'microsecond', '..."
1,1,Decoding T-SQL CAST in C#/VB.NET,<p>Recently our site has been deluged with the...,<c#><sql><vb.net><ascii><hex>,109,68,6252,2,"['c#', 'sql', 'vb.net', 'ascii', 'hex']","['recently', 'site', 'deluge', 'resurgence', '...","['decode', 'sql', 'cast']"
2,2,Displaying Flash content in a C# WinForms appl...,<p>What is the best way to display <code>Flash...,<c#><winforms><flash><adobe><macromedia>,1037,40,37517,2,"['c#', 'winforms', 'flash', 'adobe', 'macromed...","['good', 'way', 'display', 'flash', 'content',...","['display', 'flash', 'content', 'winforms', 'a..."
3,3,Why doesn't SQL Full Text Indexing return resu...,"<p>For instance, my query is like the followin...",<sql><sql-server><sql-server-2005><indexing><f...,1042,26,9479,2,"['sql', 'sql-server', 'sql-server-2005', 'inde...","['instance', 'query', 'like', 'follow', 'sql',...","['sql', 'text', 'indexing', 'return', 'result'..."
4,4,How can I unit test Flex applications from wit...,<p>I'm currently working on an application wit...,<apache-flex><eclipse><unit-testing><build-aut...,2222,18,2764,4,"['apache-flex', 'eclipse', 'unit-testing', 'bu...","['currently', 'work', 'application', 'frontend...","['unit', 'test', 'flex', 'application', 'ide',..."


In [44]:
def convert_columns(text):
    """
    Convertit une chaîne de caractères en liste de mots en remplaçant certains caractères spéciaux et en séparant les éléments.

    Args:
        text (str): La chaîne de caractères à traiter. Celle-ci peut contenir des crochets, des virgules et des guillemets.

    Returns:
        list: Une liste de mots extraits de la chaîne, avec les caractères spéciaux remplacés par des espaces.
    """
    return text.replace("[", " ").replace("]", " ").replace(",", " ").replace("'", " ").split()

In [45]:
# Appliquer la fonction 'convert_columns' aux variables 'preproc_tags', 'preproc_body', 'preproc_title'

df['preproc_title'] = df['preproc_title'].apply(convert_columns)
df['preproc_body'] = df['preproc_body'].apply(convert_columns)
df['preproc_tags'] = df['preproc_tags'].apply(convert_columns)

In [46]:
# Fonction pour le pre-processing du texte

# Charger le modèle de langue
nlp = spacy.load("en_core_web_sm")

# Liste des mots spécifiques à conserver (comme "C++", "C#", etc.)
whitelist = ["c#", "c++"]

# Fonction pour le traitement du texte
def preprocess_text(text):
    # Enlever les balises HTML
    #soup = BeautifulSoup(text, "lxml").get_text()
    doc = nlp(text)  # Traiter le texte avec spaCy
    
    # Expression régulière pour vérifier les caractères anglais uniquement (ASCII)
    def is_english(token):
        return re.match(r'^[a-zA-Z0-9+.#]+$', token)

    tokens = [
        # Utiliser le lemme sauf si le mot est dans la whitelist
        token.lemma_.lower() if token.lemma_.lower() not in whitelist else token.text.lower()
        for token in doc 
        if not token.is_stop                         # Ne pas inclure les stopwords
        and not token.is_punct                       # Ne pas inclure la ponctuation
        and not token.like_num                       # Ne pas inclure les chiffres
        and len(token.lemma_) >= 3                    # Exclure les tokens trop courts
        and (is_english(token.text) or token.text.lower() in whitelist)  # Garder les mots anglais ou ceux de la whitelist
    ]
    print(tokens)
    return tokens

## B. Features Engineering

In [47]:
df['encoded_tags'] = LabelEncoder().fit_transform(df['preproc_tags'].apply(lambda x: ' '.join(x)))

In [48]:
# Creation de la variable 'text' qui regroupe les colonnes 'preproc_title' et 'preproc_body'

df['corpus'] = df['preproc_body'].apply(lambda x: ' '.join(x)) + ' ' + df['preproc_title'].apply(lambda x: ' '.join(x))

In [49]:
# Créer une liste de tous les tags présents dans la variable 'preproc_tags'
all_tags = df['preproc_tags'].explode().tolist()
# BEGIN: LabelEncoder for all_tags
label_encoder = LabelEncoder()
encoded_label = label_encoder.fit_transform(all_tags)

## C. Modelisation

### 1 : Regression Logistique avec Bag of Words

#### Creation du Bag-Of-Words

In [50]:
# Initialiser le CountVectorizer
vectorizer = CountVectorizer()

# Appliquer le vectorizer sur le corpus
X_bow = vectorizer.fit_transform(df['corpus'].sample(50000))

# Afficher la forme de la matrice BoW
print(X_bow.shape)

(50000, 118148)


: 

#### Reduction des dimensions avec ACP

In [37]:
#Appliquer l'ACP pour réduire la dimensionnalité
pca = PCA(n_components=2)  # Réduire à 2 dimensions pour la visualisation
X_pca = pca.fit_transform(X_bow.toarray())  # Convertir en tableau dense

# Créer un DataFrame pour stocker les résultats de l'ACP
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])

#### Separation des donnees en ensemble d'entrainement et de test

In [38]:
# Répartition des données
X_train, X_test, y_train, y_test = train_test_split(X_pca, encoded_label[:50000], test_size=0.2, random_state=42)

#### Modelisation

In [39]:
# Modelisation avec Regression Logistique
# Initialiser le classifieur
model_reg_log = LogisticRegression(C=0.1, max_iter=100, solver='saga')

# Entrainer GridSearchCV
model_reg_log.fit(X_train, y_train)

# Faire des predictions sur l'ensemble de test
y_pred = model_reg_log.predict(X_test)

/Users/vdumont/Library/Caches/pypoetry/virtualenvs/projet_5-xoaF25Sd-py3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


#### Resultat & Scoring

#### Predire une suggestion de tags

In [40]:
phrase = "How to compile with gcc on Debian 10 ?"

phrase_vectorizer = vectorizer.fit_transform(preprocess_text(phrase))
phrase_vectorizer_acp = pca.fit_transform(phrase_vectorizer.toarray())
predicted_tags = model_reg_log.predict(phrase_vectorizer_acp)

# Afficher les tags encodés dans leur format textuel
decoded_tags = label_encoder.inverse_transform(predicted_tags)
print(decoded_tags)

['compile', 'gcc', 'debian']
['java' 'c#' 'java']


### 2 : Regression Logistique avec Word2Vec

#### Creation de vecteurs avec Word2Vec

#### Separation des donnees en ensemble d'entrainement et de test

#### Modelisation

#### Resultat & Scoring

#### Predire une suggestion de tags